In [ ]:
!pip install tslearn
!pip install ydata-synthetic
!pip install tsaug

In [100]:
from keras import *
import os
import numpy as np
import tensorflow as tf
import random as rn
import pandas as pd
from sklearn.model_selection import train_test_split
from tsaug.visualization import plot
from tsaug import TimeWarp, Crop, Quantize, Drift, Reverse
from tensorflow.keras.constraints import max_norm
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, AveragePooling1D, LSTM, Conv1D, Flatten, SeparableConv2D, Dropout, DepthwiseConv2D, MaxPooling2D, Activation, BatchNormalization, MaxPooling1D, Softmax, Input, Conv2D, AveragePooling2D
from numpy import genfromtxt
import scipy.io
import pandas as pd
import importlib
import sys  
from sklearn.preprocessing import MinMaxScaler
import sklearn.cluster as cluster
import matplotlib.pyplot as plt
from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans
from ydata_synthetic.synthesizers.timeseries import TimeGAN

os.environ['PYTHONHASHSEED']='0'
np.random.seed(1)
rn.seed(2)

#### READ DATA

In [101]:
mat = scipy.io.loadmat('subj1_train1.mat')
trials = np.swapaxes(mat['data'],0,2)
trials = np.swapaxes(trials,1,2)

mat1 = scipy.io.loadmat('subj1_train2.mat')

trials1 = np.swapaxes(mat1['data'],0,2)
trials1 = np.swapaxes(trials1,1,2)

mat2 = scipy.io.loadmat('subj2_train1.mat')

trials2 = np.swapaxes(mat2['data'],0,2)
trials2 = np.swapaxes(trials2,1,2)

mat3 = scipy.io.loadmat('subj2_train2.mat')

trials3 = np.swapaxes(mat3['data'],0,2)
trials3 = np.swapaxes(trials3,1,2)

trials = np.concatenate([trials, trials1, trials2, trials3])

labels_map = {0: '9Hz', 1: '10Hz', 
                2: '12Hz', 3: '15Hz'}

alt_labels_map = {0: 'TOP', 1: 'RIGHT', 
                2: 'BOTTOM', 3: 'LEFT'}

def read_labels (file_path, labels_map):
    oneHot_labels=genfromtxt(file_path,delimiter=' ')
    labels = np.argmax(oneHot_labels, axis=1)

    return labels, [labels_map[idx] for idx in labels]

labels = read_labels('classInfo_4_5.m', labels_map)

lab = labels[0].tolist()
for x in labels[0]:
  lab.append(x)
  lab.append(x)
  lab.append(x)

In [ ]:
plot(trials[0])

#### TIMESERIES AUGMENTATION
Compute 10 new trials from every trial.

TODO:


*   Test other timeseries augmenter 




In [ ]:
my_augmenter = (
TimeWarp() * 15  # random time warping 15 times in parallel
+ Crop(size=1882)  # random crop subsequences with length 300
+ Quantize(n_levels=[10, 20, 30])  # random quantize to 10-, 20-, or 30- level sets
+ Drift(max_drift=(0.1, 0.5)) @ 0.8  # with 80% probability, random drift the signal up to 10% - 50%
+ Reverse() @ 0.5  # with 50% probability, reverse the sequence
)
trials_synt = []
labels_synt = []
for x in range(len(trials)):
  i = my_augmenter.augment(trials[x])
  ls1 = []
  for t in range(i.shape[0]):
    ls1.append(i[t])
    if len(ls1) == 8:
      trials_synt.append(ls1)
      ls1 = []
      labels_synt.append(lab[x])

trials_synt = np.array(trials_synt)
trials_synt.shape

In [ ]:
plot(trials_synt[0])

#### RESHAPE AND CREATE TRAIN/TEST DATA FOR CNN

In [105]:
x_trials_synt = np.resize(trials_synt, (1200, 8, 1882))
y_train = to_categorical(labels_synt)


trials1 = np.resize(trials, (80, 8, 1882))
x_train_ft, x_test_ft, y_train_ft, y_test_ft = train_test_split(trials1, lab, test_size= 0.2, random_state=42)
y_test_ft = to_categorical(y_test_ft)
y_train_ft = to_categorical(y_train_ft)

#### DEFINE, TRAIN, FINE-TUNE, TEST OF CNN
TODO:


*   Hyperparameters search
*   Different architectures



In [ ]:
# DEFINE HYPERPARAMETERS FOR CLASSIFICATION
nb_classes = 4
Chans = 8
Samples = 1882 
dropoutRate = 0.5
kernLength = 10
F1 = 256 
D = 1
F2 = 256
dropoutType = Dropout

#DEFINE MODEL

input1   = Input(shape = (8, 1882, 1))
block1  = Conv2D(F1, (1, kernLength), padding = 'same',
                  input_shape = (Chans, Samples, 1))(input1)
block1 = BatchNormalization()(block1)
block1 = DepthwiseConv2D((Chans, 1), use_bias = False, 
                          depth_multiplier = D,
                          depthwise_constraint = max_norm(1.))(block1)
block1 = BatchNormalization()(block1)
block1= Activation('elu')(block1)
block1= AveragePooling2D((1, 4))(block1)
block1= dropoutType(dropoutRate)(block1)
    
block2= SeparableConv2D(F2, (1, 16), padding = 'same')(block1)
block2= BatchNormalization()(block2)
block2= Activation('elu')(block2)
block2= AveragePooling2D((1, 8))(block2)
block2= dropoutType(dropoutRate)(block2)
        
flatten= Flatten(name = 'flatten')(block2)
dense1 = Dense(600, name = 'dense1')(flatten)

dense= Dense(nb_classes, name = 'dense')(dense1)
softmax= Activation('softmax', name = 'softmax')(dense)
    
model = Model(inputs=input1, outputs=softmax)

#COMPILE MODEL
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

#TRAIN ON SYNTHETIC DATA
model.fit(trials_synt, y_train, epochs = 40, batch_size=16)
model.evaluate(x_test_ft, y_test_ft)

#FINE-TUNE ON TRIAL DATA
model.fit(x_train_ft, y_train_ft, epochs = 10, batch_size=1)
model.evaluate(x_test_ft, y_test_ft)

#### SCALE DATA TO [-1, 1]

TODO:


*   Different scalers



In [107]:
x_train = np.resize(x_trials_synt, [800, 1882, 8])
ls = []
for x in range(len(x_train1)):
  scaler = MinMaxScaler()
  train_data = scaler.fit_transform(x_train1[x])
  ls.append(train_data)

x_train_gan = np.array(ls)

#### DEFINE HYPERPARAMETERS FOR TIMEGAN
TODO:

*   Hyperparameters search



In [108]:
model = TimeGAN

seq_len = 1882
n_seq = 8
noise_dim = 8
dim = 16
batch_size = 256
log_step = 100
learning_rate = 5e-4

gan_args = [batch_size, learning_rate, noise_dim, 1882, 2, (0, 1), dim]
models_dir = './cache'
synthesizer = model(gan_args, hidden_dim=8, seq_len=seq_len, n_seq=n_seq, gamma=0.1)

#### TRAIN TIMEGAN

In [ ]:
synthesizer.train(x_train_gan, train_steps=10000)

#### SYNTHETIZE NEW DATA FROM TIMEGAN

In [ ]:
data_sample = synthesizer.sample(10000)
data_sample[0]

In [ ]:
data_sample1 = []
for x in data_sample:
  data_sample1.append(scaler.inverse_transform(x))
data_sample1[0]

In [ ]:
data_sample1[0].shape

#### CREATE LABELS FOR SYNTHETIC DATA

In [ ]:
km = TimeSeriesKMeans(n_clusters=4, metric="dtw", max_iter=100,
                       random_state=0).fit(trials)
pred = km.predict(data_sample1)
pred

In [ ]:
test = np.resize(data_sample1[0], (8, 1882))
plot(test[0])

#### PREPARE DATA FOR CNN

In [ ]:
trials_synt = np.resize(data_sample1, (10004, 8, 1882))
y_train_synt = to_categorical(pred)

trials1 = np.resize(trials, (80, 8, 1882))
x_train_ft, x_test_ft, y_train_ft, y_test_ft = train_test_split(trials1, lab, test_size= 0.2, random_state=42)
y_test_ft = to_categorical(y_test_ft)
y_train_ft = to_categorical(y_train_ft)

####DEFINE, TRAIN, FINE-TUNE, TEST CNN

TODO:


*   Search for hyperparameters



In [ ]:
nb_classes = 4
Chans = 8
Samples = 1882 
dropoutRate = 0.5
kernLength = 10
F1 = 256 
D = 1
F2 = 256
dropoutType = Dropout

input1   = Input(shape = (8, 1882, 1))

##################################################################
block1 = Conv2D(F1, (1, kernLength), input_shape = (Chans, Samples, 1))(input1)
block1 = BatchNormalization()(block1)
block1 = DepthwiseConv2D((Chans, 1), use_bias = False, 
                          depth_multiplier = D,
                          depthwise_constraint = max_norm(1.))(block1)
block1 = BatchNormalization()(block1)
block1= Activation('elu')(block1)
block1= AveragePooling2D((1, 4))(block1)
block1= dropoutType(dropoutRate)(block1)
    
block2= SeparableConv2D(F2, (1, 16), padding = 'same')(block1)
block2= BatchNormalization()(block2)
block2= Activation('elu')(block2)
block2= AveragePooling2D((1, 8))(block2)
block2= dropoutType(dropoutRate)(block2)
        
flatten= Flatten(name = 'flatten')(block2)
dense1 = Dense(1000, name = 'dense1')(flatten)

dense= Dense(nb_classes, name = 'dense')(dense1)
softmax= Activation('softmax', name = 'softmax')(dense)
    
model = Model(inputs=input1, outputs=softmax)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()
model.fit(trials_synt, y_train, epochs = 1, batch_size=128)
model.evaluate(x_test_ft, y_test_ft)
model.fit(x_train_ft, y_train_ft, epochs = 10, batch_size=1)
model.evaluate(x_test_ft, y_test_ft)

In [ ]:

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn, 
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        x, y = data
        teacher_predictions = self.teacher(x, training=False)
   
        with tf.GradientTape() as tape:
            student_predictions = self.student(x, training=True)
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
             )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(y, student_predictions)

        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        x, y = data
        y_prediction = self.student(x, training=False)
        student_loss = self.student_loss_fn(y, y_prediction)

        self.compiled_metrics.update_state(y, y_prediction)

        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results